#### ALO 선언하기 및 TCR setup 하기 
📜 Titanic : http://mod.lge.com/hub/dxadvtech/aicontents/titanic.git    
📜 TCR : http://mod.lge.com/hub/dxadvtech/aicontents/tcr.git   

ALO 의 requirements.txt 를 설치 합니다. 

In [ ]:
%pip install -r requirements.txt

ALO 를 TCR 로 설정합니다. 
 -  alo.init("http://mod.lge.com/hub/dxadvtech/aicontents/titanic.git")

In [ ]:
from src.alo import ALO
import os 
from pprint import pprint

alo = ALO()
alo.init("http://mod.lge.com/hub/dxadvtech/aicontents/titanic.git")
alo.set_asset_structure()

Data path 를 변경 합니다. 

In [ ]:
alo.experimental_plan.external_path['load_train_data_path'] = "../../sample_data/default_train/"
alo.experimental_plan.external_path['load_inference_data_path'] = "../../sample_data/default_inference/"
pprint(alo.experimental_plan.external_path)


Train pipeline 을 실행 합니다.

In [ ]:
# alo.asset_source['train_pipeline']

alo.system_envs['pipeline_list'] = ['train_pipeline']
alo.external_load_data('train_pipeline')
alo.install_steps('train_pipeline', 'once')
# alo.run('train_pipeline')
alo.runs()


Inference pipeline 을 실행 합니다.

In [ ]:
alo.system_envs['pipeline_list'] = ['inference_pipeline']
# alo.external_load_data('inference_pipeline')
# alo.install_steps('inference_pipeline', 'once')
alo.runs()

Inferece 결과를 외부에 저장 합니다. 

In [ ]:
alo.experimental_plan.external_path['save_inference_artifacts_path'] = '../../serving_test'
display(alo.experimental_plan.external_path)
alo.runs()

In [ ]:
# alo asset의 설치하기 위한 정보를 asset_config라는 변수에 담는다
asset_config = alo.asset_source['train_pipeline']
asset_config

In [ ]:
# train pipeline에서 externel data path에 입력되어 있는 데이터를 가지고 온다
alo.external_load_data('train_pipeline')

In [ ]:
# train pipeline의 asset들을 설치 한다(once : 이미 설치되어 있으면 설치 하지 않는다, every : 설치가 되어 있어도 다시 설치 한다)
# experimental_plan.yaml의 control의 get_asset_source 키의 값을 사용한다
alo.install_steps('train_pipeline', 'once')

In [ ]:
# 입력 받은 asset을 동작시킨다. 첫 번째 step (0번 step) 을 구동
alo.asset_structure = alo.process_asset_step(asset_config[0], 0, "train_pipeline", alo.asset_structure)

In [ ]:
# 1번 step의 args를 받아야 한다
alo.asset_structure.args = alo.get_args("train_pipeline", 1)
alo.asset_structure.args

In [ ]:
# 1번 step의 asset을 실행한다
alo.process_asset_step(asset_config[1], 1, "train_pipeline", alo.asset_structure)

In [ ]:
display(alo.experimental_plan.external_path)
alo.experimental_plan.external_path['save_inference_artifacts_path'] = '../test'
display(alo.experimental_plan.external_path)